In [1]:
import numpy as np
import pandas as pd

In [2]:
%run 0.1_Variables-and-Constants.ipynb

In [3]:
def result_df(__type="成績",__start="2015", __end="2025", __location=location_list):
    csv_dir = Path(base_dir, __type)
    csv_files = csv_dir.glob("**/*.CSV")

    # 空のDataFrameを作成
    df = pd.DataFrame()
    
    # カラム名
    if __type == "成績":
        columns = result_columns
    elif __type == "基準タイム":
        columns = time_columns
    elif __type == "配当":
        columns = payout_columns
    else:
        raise Exception
        

    # CSVファイルごとに処理
    for file in csv_files:
        year = "20" + file.stem[0:2]
        held = file.stem[2]
        location = file.stem[3]
        if (
            int(year) >= int(__start)
            and int(year) <= int(__end)
            and location in __location
        ):
            # 最適化: chunksizeを指定してメモリ効率良く処理
            chunk_iter = pd.read_csv(
                file,
                chunksize=10000,
                encoding="cp932",
                header=None,
                names=columns,
            )
            for chunk in chunk_iter:
                df = pd.concat([df, chunk], ignore_index=True)
    return df

In [4]:
df = result_df(__type='配当',__start="2021", __end="2024", __location=['東'])

In [12]:
#df.columns
df[['レースID','単勝配当1','複勝配当1', '複勝配当2','複勝配当3','ワイド配当1','ワイド配当2','ワイド配当3', '馬連配当1','馬単配当1','３連複配当1','３連単配当1']]

,レースID,単勝配当1,複勝配当1,複勝配当2,複勝配当3,ワイド配当1,ワイド配当2,ワイド配当3,馬連配当1,馬単配当1,３連複配当1,３連単配当1
0,5221101,790,200,180,200,580,760,540,1560,3500,3450,19720
1,5221102,400,130,110,200,180,700,450,350,890,1420,6330
2,5221103,19070,3030,240,380,12970,20620,1500,65370,199920,209060,2475070
3,5221104,9120,1660,2380,180,42750,5130,3590,267610,574380,314170,3668310
4,5221105,320,120,110,110,240,150,160,930,1650,360,3530
...,...,...,...,...,...,...,...,...,...,...,...,...
1603,05222C08,300,120,140,140,220,330,410,510,940,1110,3770
1604,05222C09,630,240,360,180,1240,720,1160,4090,7180,5570,36810
1605,05222C10,1360,310,140,200,730,1700,460,1820,4670,3790,27120
1606,05222C11,420,160,150,410,340,1120,1390,730,1440,4570,15770


In [4]:
def time_df(t_start="2015", t_end="2025", t_location=location_list):
    csv_dir = Path(time_dir)
    csv_files = csv_dir.glob("**/*.CSV")

    # 空のDataFrameを作成
    df = pd.DataFrame()

    # CSVファイルごとに処理
    for file in csv_files:
        year = "20" + file.stem[0:2]
        held = file.stem[2]
        location = file.stem[3]
        if (
            int(year) >= int(t_start)
            and int(year) <= int(t_end)
            and location in t_location
        ):
            # 最適化: chunksizeを指定してメモリ効率良く処理
            chunk_iter = pd.read_csv(
                file,
                chunksize=10000,
                encoding="cp932",
                header=None,
                names=time_columns,
            )
            for chunk in chunk_iter:
                df = pd.concat([df, chunk], ignore_index=True)
    pattern = r"(?P<min>\d)\.(?P<sec>\d{2})\.(?P<tenth>\d)"
    out = df["基準タイム"].str.extract(pattern).astype(int)
    df["基準タイム(秒)"] = out["min"] * 60 + out["sec"] + out["tenth"] / 10
    return df